# Fine-tuning de GPT-2 com PEFT (LoRA) no Dataset AmazonTitles-1.3MM

Este notebook demonstra o processo de fine-tuning do modelo GPT-2 usando o método PEFT (Parameter-Efficient Fine-Tuning) com LoRA (Low-Rank Adaptation) no dataset AmazonTitles-1.3MM. Utilizaremos as bibliotecas Hugging Face Transformers, Datasets e PEFT para este processo.

## Instalação das Dependências

Primeiro, vamos instalar as bibliotecas necessárias.

In [ ]:
!pip install transformers datasets torch peft accelerate

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importação das Bibliotecas

In [17]:
import torch
from datasets import load_dataset, load_from_disk
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from accelerate import Accelerator

## Carregamento e Preparação do Dataset

Nesta seção, carregaremos o dataset AmazonTitles-1.3MM e o prepararemos para o treinamento.

In [11]:
# Carregando o dataset
# Nota: Você precisará fazer upload do arquivo trn.json para o Colab ou usar um link para o arquivo
dataset = load_dataset('json', data_files={'train': '/content/drive/MyDrive/fiap/tech-challenge-03/trn.json'})

# Função para preparar os dados
def prepare_data(examples):
    return {
        'text': [f"Title: {title}\nContent: {content}\nResponse:" for title, content in zip(examples['title'], examples['content'])]
    }

# Aplicando a preparação dos dados
dataset = dataset.map(prepare_data, batched=True, remove_columns=dataset['train'].column_names)

# Dividindo o dataset em treino e validação
dataset = dataset['train'].train_test_split(test_size=0.1)

print(f"Tamanho do conjunto de treinamento: {len(dataset['train'])}")
print(f"Tamanho do conjunto de validação: {len(dataset['test'])}")

Tamanho do conjunto de treinamento: 2023757
Tamanho do conjunto de validação: 224862


## Configuração do Tokenizador

In [32]:
# Configurando o tokenizador
model_name = 'gpt2'  # Usando o modelo menor 'gpt2' em vez de 'gpt2-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

# Função para tokenizar os dados
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# Aplicando a tokenização
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])


## Salvando dataset tokenizado

In [14]:
tokenized_dataset_save_path ="/content/drive/MyDrive/fiap/tech-challenge-03/tokenized_dataset"
tokenized_datasets.save_to_disk(tokenized_dataset_save_path)
print("Dataset tokenizado salvo em:", tokenized_dataset_save_path)


Saving the dataset (0/6 shards):   0%|          | 0/2023757 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/224862 [00:00<?, ? examples/s]

Dataset tokenizado salvo em: /content/drive/MyDrive/fiap/tech-challenge-03/tokenized_dataset



## Carregar o dataset tokenizado


In [33]:
tokenized_dataset_save_path ="/content/drive/MyDrive/fiap/tech-challenge-03/tokenized_dataset"
tokenized_dataset = load_from_disk(tokenized_dataset_save_path)
# Preparando o data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Configuração do Modelo e PEFT com LoRA

In [36]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Configuração do LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=6,
    lora_alpha=32,
    lora_dropout=0.1,
    fan_in_fan_out=True,
)

# Aplicar PEFT ao modelo
peft_model = get_peft_model(model, peft_config)

## Configuração do Treinamento

In [37]:
# Configuração otimizada do treinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Reduzindo o número de épocas
    per_device_train_batch_size=32,  # Aumentando o tamanho do batch
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    fp16=True,  # Habilitando treinamento em precisão mista
    gradient_accumulation_steps=2,
    learning_rate=5e-4,  # Aumentando a taxa de aprendizado
    report_to="none",
    max_steps=100,  # Aumentando o número máximo de passos
)

# Inicializando o Accelerator
#accelerator = Accelerator()

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)

## Treinamento do Modelo

In [38]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=100, training_loss=4.163296508789062, metrics={'train_runtime': 28.7715, 'train_samples_per_second': 222.442, 'train_steps_per_second': 3.476, 'total_flos': 838308829593600.0, 'train_loss': 4.163296508789062, 'epoch': 0.003162405325490568})

## Salvando o Modelo

In [39]:
peft_model_path = "/content/drive/MyDrive/fiap/tech-challenge-03/peft_model"
trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('/content/drive/MyDrive/fiap/tech-challenge-03/peft_model/tokenizer_config.json',
 '/content/drive/MyDrive/fiap/tech-challenge-03/peft_model/special_tokens_map.json',
 '/content/drive/MyDrive/fiap/tech-challenge-03/peft_model/vocab.json',
 '/content/drive/MyDrive/fiap/tech-challenge-03/peft_model/merges.txt',
 '/content/drive/MyDrive/fiap/tech-challenge-03/peft_model/added_tokens.json')

## Carregando o Modelo para Inferência

In [41]:
def load_peft_model(model_path):
    # Load the PEFT configuration from your saved model path, not 'gpt2'
    model = PeftModel.from_pretrained(GPT2LMHeadModel.from_pretrained('gpt2'), model_path)
    return model

peft_model_path = "/content/drive/MyDrive/fiap/tech-challenge-03/peft_model"
model = load_peft_model(peft_model_path)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=6, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=6, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
       

## Função para Gerar Respostas

In [42]:
def generate_response(title, content):
    input_text = f"Title: {title}\nContent: {content}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    with torch.no_grad():
        output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response = generated_text.split('Response:')[-1].strip()

    return response

# Exemplo de uso
title = "Smartphone de última geração"
content = "Este smartphone possui câmera de alta resolução, bateria de longa duração e processador potente."
response = generate_response(title, content)
print(f"Título: {title}")
print(f"Conteúdo: {content}")
print(f"Resposta gerada: {response}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Título: Smartphone de última geração
Conteúdo: Este smartphone possui câmera de alta resolução, bateria de longa duração e processador potente.
Resposta gerada: response:Response : Response  Response  response response   Reply  reply  replies  responses  responds  respond  responding  responded  replied respond  answered  answers  reactions  reaction  rebuttals  rebutters  retorts  re-reply  resubmissions  reposts  comments  submissions  requests  request  requests  rejections  rejects  rejected  rejection  rejecting  refusing  refuse  refusal


## Conclusão

Este notebook demonstrou o processo de fine-tuning do modelo GPT-2 usando PEFT com LoRA no dataset AmazonTitles-1.3MM. O modelo treinado pode ser usado para gerar respostas baseadas em títulos e descrições de produtos.

Pontos importantes:
1. PEFT com LoRA permite um fine-tuning eficiente em termos de memória e computação.
2. O modelo treinado mantém o conhecimento geral do GPT-2 enquanto se adapta à tarefa específica.
3. O processo de treinamento é mais rápido e requer menos recursos comparado ao fine-tuning tradicional.
4. O modelo resultante é menor em tamanho, facilitando o armazenamento e a distribuição.

Para melhorar ainda mais:
- Experimente com diferentes configurações de LoRA (r, alpha, dropout).
- Ajuste os hiperparâmetros de treinamento para otimizar o desempenho.
- Considere usar técnicas de avaliação mais robustas para medir a qualidade das respostas geradas.